In [1]:
%run gower_function-v6.4.ipynb
from pymongo import MongoClient
from random import randint
import pandas as pd
import numpy as np
import re
import category_encoders as ce
from scipy.spatial.distance import pdist,squareform, jaccard, cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
import category_encoders as ce

In [103]:
client = MongoClient(port=27017)
db = client.weare    

In [119]:
student_queries = db.students.find({})
user_queries = db.users.find({})
students = pd.DataFrame(list(student_queries))
users = pd.DataFrame(list(user_queries))
users.columns

Index(['_id', 'channels', 'email', 'first_name', 'group', 'image_48', 'is_bot',
       'is_custom_image', 'last_name', 'last_updated', 'local_area', 'locale',
       'name', 'phone', 'real_name', 'status_emoji', 'status_expiration',
       'status_text', 'team_id', 'title', 'tz', 'tz_label', 'tz_offset',
       'uid'],
      dtype='object')

In [120]:
def clean_channels(channel_obj_array):
    channels = []
    if isinstance(channel_obj_array, str):
        channel_obj_array = json.loads(channel_obj_array) 
    for channel_obj in channel_obj_array:
        if isinstance(channel_obj, collections.Mapping):
            channels.append(channel_obj['cid'])
            continue
        if isinstance(channel_obj, str):
            channels.append(channel_obj)
            continue
        raise Exception('Invalid type')
    return channels
users.channels = users.channels.apply(clean_channels)

In [121]:
binarized_channels = users.channels.apply('|'.join).str.get_dummies()
col_list = list(binarized_channels.columns)
column_mappings = {name: f'channel_{i}' for i, name in enumerate(col_list)}
binarized_channels = binarized_channels.rename(index=str, columns=column_mappings)
binarized_channels = 

In [122]:
binarized_channels

,channel_0,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,channel_9,...,channel_14,channel_15,channel_16,channel_17,channel_18,channel_19,channel_20,channel_21,channel_22,channel_23
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,0,1,1,0,...,0,1,0,1,1,0,0,0,0,0
7,1,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
8,1,1,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0


In [143]:
pd.merge(users, binarized_channels.reset_index(), left_index=True, right_index=True)

,_id,channels,email,first_name,group,image_48,is_bot,is_custom_image,last_name,last_updated,...,channel_14,channel_15,channel_16,channel_17,channel_18,channel_19,channel_20,channel_21,channel_22,channel_23
0,ObjectId(5c399149ff0b61b77d00ce4d),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",bvh10@ist.psu.edu,Ben,3,https://secure.gravatar.com/avatar/28802966d0f...,False,None,Hanrahan,1504319350,...,0,0,0,0,0,0,0,0,0,0
1,ObjectId(5c399149ff0b61b77d00ce4e),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",yuw132@psu.edu,wuyu,2,https://secure.gravatar.com/avatar/6349a73089d...,False,None,None,1504319350,...,0,0,0,0,0,0,0,0,0,0
2,ObjectId(5c399149ff0b61b77d00ce4f),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",tuy11@psu.edu,Tina,3,https://secure.gravatar.com/avatar/e74e1765013...,False,None,None,1504319351,...,0,0,0,0,0,0,0,0,0,0
3,ObjectId(5c399149ff0b61b77d00ce50),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",jomarabinda@gmail.com,jomarabinda,4,https://secure.gravatar.com/avatar/7392ec92f00...,False,None,None,1504319350,...,0,0,0,0,0,0,0,0,0,0
4,ObjectId(5c399149ff0b61b77d00ce55),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",dxc360@ist.psu.edu,Dong,1,https://secure.gravatar.com/avatar/2434caac082...,False,None,Chen,1504319351,...,0,0,0,0,0,0,0,0,0,0
5,ObjectId(5c399149ff0b61b77d00ce56),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",adithya@pressmail.ch,adithyavr,1,https://secure.gravatar.com/avatar/8ef0409aac3...,False,None,None,1504319351,...,0,0,0,0,0,0,0,0,0,0
6,ObjectId(5c399149ff0b61b77d00ce58),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",jop5190@ist.psu.edu,joslenne,1,https://avatars.slack-edge.com/2018-10-12/4547...,False,True,,1542311853,...,0,1,0,1,1,0,0,0,0,0
7,ObjectId(5c399149ff0b61b77d00ce59),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",cya5092@ist.psu.edu,top,4,https://secure.gravatar.com/avatar/219cb99e21d...,False,None,None,1504319352,...,1,0,0,0,0,0,1,0,0,0
8,ObjectId(5c399149ff0b61b77d00ce5b),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",jess.kropczynski@gmail.com,jesskropczynski,2,https://secure.gravatar.com/avatar/c8ff9bb4f8a...,False,None,None,1504319366,...,0,0,0,0,0,0,0,0,0,0
9,ObjectId(5c399149ff0b61b77d00ce5e),"[T0A286J8K_C0A28BAHG, T0A286J8K_C0A28CE5P, T0A...",bikalpaneupane@ist.psu.edu,Bikalpa,2,https://secure.gravatar.com/avatar/11192bc82cc...,False,None,None,1504319351,...,0,0,1,0,0,0,0,0,0,0


In [118]:
users = pd.concat([users, binarized_channels], axis=1, sort=False)
users = users.drop('channels', 1)
users

,_id,email,first_name,group,image_48,is_bot,is_custom_image,last_name,last_updated,local_area,...,channel_14,channel_15,channel_16,channel_17,channel_18,channel_19,channel_20,channel_21,channel_22,channel_23
0,ObjectId(5c399149ff0b61b77d00ce4d),bvh10@ist.psu.edu,Ben,3,https://secure.gravatar.com/avatar/28802966d0f...,False,None,Hanrahan,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ObjectId(5c399149ff0b61b77d00ce4e),yuw132@psu.edu,wuyu,2,https://secure.gravatar.com/avatar/6349a73089d...,False,None,None,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ObjectId(5c399149ff0b61b77d00ce4f),tuy11@psu.edu,Tina,3,https://secure.gravatar.com/avatar/e74e1765013...,False,None,None,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ObjectId(5c399149ff0b61b77d00ce50),jomarabinda@gmail.com,jomarabinda,4,https://secure.gravatar.com/avatar/7392ec92f00...,False,None,None,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ObjectId(5c399149ff0b61b77d00ce55),dxc360@ist.psu.edu,Dong,1,https://secure.gravatar.com/avatar/2434caac082...,False,None,Chen,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ObjectId(5c399149ff0b61b77d00ce56),adithya@pressmail.ch,adithyavr,1,https://secure.gravatar.com/avatar/8ef0409aac3...,False,None,None,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ObjectId(5c399149ff0b61b77d00ce58),jop5190@ist.psu.edu,joslenne,1,https://avatars.slack-edge.com/2018-10-12/4547...,False,True,,1.542312e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ObjectId(5c399149ff0b61b77d00ce59),cya5092@ist.psu.edu,top,4,https://secure.gravatar.com/avatar/219cb99e21d...,False,None,None,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ObjectId(5c399149ff0b61b77d00ce5b),jess.kropczynski@gmail.com,jesskropczynski,2,https://secure.gravatar.com/avatar/c8ff9bb4f8a...,False,None,None,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ObjectId(5c399149ff0b61b77d00ce5e),bikalpaneupane@ist.psu.edu,Bikalpa,2,https://secure.gravatar.com/avatar/11192bc82cc...,False,None,None,1.504319e+09,America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
students['id']=students.Email.str.split("@", n=1, expand=True)[0]
if '_id' in students.columns:
    del students['_id']

In [30]:
#drop rows with 5 or more null values
students.dropna(thresh=5, inplace=True)
students = students.replace('', np.nan)

In [31]:
#number of courses, credits are too dirty as many entered text instead of a number
drop_columns = ["surveyCompletion", "Duration (in seconds)", "NumCourses", "transferCredits", "NumTranCredits",
                "WhyProfile", "otherChannels", "otherEmploy", "NonUS", "otherIndusry", "KnownThroughProfile",
                "otherEth", "PPLinPerson", "otherEmail"]
#only drop these columns if they actually exist in the data
drop_columns = [col for col in drop_columns if col in students.columns]
students.drop(columns=drop_columns, inplace=True)
#names of all the soc columns
SOCs = ['SOC'+str(x+1) for x in range(10)]
#names of all the CCE columns
CCEs = ['CCE'+str(x+1) for x in range(24)]
#mapping from each text column to ordinal values
#format is array of dicts containing keys 'col' and 'mapping'
ordinal_cols_mapping = []

In [32]:
#the following are mappings from textual column values to numberical ones
important_scale = [
        ('Extremely important', 7),
        ('Pretty important', 6),
        ('Fairly important', 5),
        ('Moderately important', 4),
        ('Somewhat important', 3),
        ('A little important', 2),
        ('Not at all important', 1)
]

interest_scale = [
    ('Extremely interested', 5),
    ('Rather interested', 4),
    ('Somewhat interested', 3),
    ('A bit of interest', 2),
    ('Not at all interested', 1)
]

Agree_scale = [
        ('Strongly agree', 5),
        ('Somewhat agree', 4),
        ('Neither agree nor disagree', 3),
        ('Somewhat disagree', 2),
        ('Strongly disagree', 1)
]

In [33]:
#fill in the mapping array 
for SOC in SOCs:
    ordinal_cols_mapping.append({
        "col": SOC,
        "mapping": Agree_scale
    })

for CCE in CCEs:
    ordinal_cols_mapping.append({
        "col": CCE,
        "mapping": Agree_scale
    })

for PeerInfo in ["PeerAvail", "PeerProfession", "PeerEdu", "PeerDetails"]:
    ordinal_cols_mapping.append({
        "col": PeerInfo,
        "mapping": important_scale
    })

ordinal_cols_mapping.append({
    "col": "Mconnected",
    "mapping": interest_scale
})

In [34]:
encoder = ce.OrdinalEncoder(mapping=ordinal_cols_mapping, return_df=True)
df = encoder.fit_transform(students)
#Create new columns by taking the mean of related columns for each row
df['SOC_f'] = df.apply(lambda row: np.mean([row['SOC9'], row['SOC10']]), axis=1)
df['SOC_id'] = df.apply(lambda row: np.mean([row['SOC1'], row['SOC2'], row['SOC3'], row['SOC4'], row['SOC5']]), axis=1)
df['CCE_IR'] = df.apply(lambda row: np.mean([row['CCE13'], row['CCE10'], row['CCE19'], row['CCE9']]), axis=1)
df['CCE_Coor'] = df.apply(lambda row: np.mean([row['CCE24'], row['CCE20'], row['CCE22'], row['CCE21'], row['CCE17']]), axis=1)
df['CCE_SS'] = df.apply(lambda row: np.mean([row['CCE1'], row['CCE2'], row['CCE3'], row['CCE4']]), axis=1)
df.drop(columns=CCEs + SOCs, inplace=True)

In [35]:
none_i = re.compile(r'none', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
none_i = re.compile(r'zero', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
stringany = re.compile(r'[a-zA-Z ()]+', flags=re.IGNORECASE)
print(f'Before digit processing, kids are {df.kids.unique()}')
df.kids = df['kids'].replace(stringany, '')

Before digit processing, kids are [ 0.  1.  2.  3.  4. nan  5.  9.]


In [36]:
categorical_cols = ["gender", "InUS", "ethnicity", "Usstate", "marrital", "employment", "industry"]
categorical_cols = [c for c in categorical_cols if c in df.columns]
df_c_mode = df[categorical_cols].mode()
print(f'mode listed are \n{df_c_mode.iloc[0]}')
print(len(df_c_mode))
for col in categorical_cols+['kids']:
    df[col].fillna(df[col].mode().iloc[0], inplace=True)


mode listed are 
gender                         Female
InUS                              Yes
ethnicity                   Caucasian
Usstate                  Pennsylvania
marrital                      Married
employment    Working (paid employee)
industry         Educational services
Name: 0, dtype: object
1


In [37]:
# fill with mode, mean, or median
df_mode, df_mean, df_median = df.mode().iloc[0], df.mean(), df.median()
df.fillna(df_median, inplace=True)

In [38]:
numeric_columns = df._get_numeric_data().columns
col_is_categorical = [col not in numeric_columns for col in df.columns]
distances = gower_distances(df, categorical_features=col_is_categorical)

/home/matt/.conda/envs/weare/lib/python3.7/site-packages/ipykernel_launcher.py:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.


In [39]:
def get_matches(index, distances, df, num_matches=1):
    best_scores = sorted(distances[index])[1: 1+num_matches]
    best_entries = [np.where(distances[index]==score)[0][0] for score in best_scores]
    og_user = df.iloc[[index]]
    best_users = [df.iloc[[entry]] for entry in best_entries]
    all_users = [og_user, *best_users]
    concat = pd.concat(all_users)
    return concat
def display_matches(distances, df, num_matches=10):
    frame = get_matches(0, distances, df)
    for i in range(1, num_matches):
        match = get_matches(i, distances, df)
        frame = pd.concat([frame, match])
    return frame

In [40]:
display_matches(distances, df)
get_matches(0, distances, df, num_matches=10)

,ActiveDuty,BWS,Degree,Email,First Name,IP Address,InUS,Last Name,Latitude,Longitude,...,industry,kids,marrital,progress,id,SOC_f,SOC_id,CCE_IR,CCE_Coor,CCE_SS
0,NaN,No,Master degree,sxa459@psu.edu,Sana,72.94.219.150,Yes,Abdulateef,40.351395,-75.930603,...,Educational services,0.0,Married,14.0,sxa459,2.5,5.0,4.50,3.8,3.50
172,NaN,No,Master degree,deb77@psu.edu,Debra,73.187.239.247,Yes,Barshinger,39.893906,-76.557999,...,Educational services,0.0,Married,8.0,deb77,3.0,4.4,4.25,4.4,4.25
32,NaN,No,Bachelor degree,ALH19@PSU.EDU,Apryl,128.118.208.194,Yes,Hawn,40.784897,-77.847000,...,Educational services,0.0,Married,12.0,ALH19,2.0,4.0,4.00,4.0,4.00
40,NaN,No,Master degree,sab89@psu.edu,Sarah,74.193.239.88,Yes,Brown,31.134506,-93.180298,...,Educational services,2.0,Married,50.0,sab89,2.0,4.4,5.00,4.8,4.75
134,NaN,No,Master degree,cdh32@psu.edu,Carolyn,96.59.78.230,Yes,Holme,27.886703,-82.511703,...,Finance or insurance,0.0,Married,2.0,cdh32,2.0,4.2,4.50,3.4,4.50
89,NaN,No,Master degree,mlb593@psu.edu,Maria,165.189.65.30,Yes,Butters,43.020798,-89.360001,...,Educational services,0.0,Married,10.0,mlb593,1.0,3.6,4.50,4.4,3.75
93,NaN,No,Master degree,jlc672@psu.edu,Jennifer,72.69.210.247,Yes,Cogen,40.714294,-74.005997,...,"Arts, entertainment or recreation",0.0,Single/Never Married,41.0,jlc672,2.5,3.2,4.50,3.8,3.50
111,NaN,No,Master degree,skg143@psu.edu,Susan,216.15.78.209,Yes,Gair,40.657104,-75.502998,...,Educational services,0.0,Married,91.0,skg143,2.0,4.6,4.50,3.8,3.75
331,No,No,Bachelor degree,cxd415@psu.edu,Caitlin,67.183.131.121,Yes,Davis,47.025696,-122.796700,...,Educational services,0.0,Married,29.0,cxd415,2.5,4.2,4.75,4.4,4.50
346,NaN,No,Bachelor degree,dmd6002@psu.edu,Denise,107.77.202.214,Yes,Dollarton,39.735107,-75.668404,...,Educational services,4.0,Single/Never Married,6.0,dmd6002,1.0,3.8,4.50,4.6,3.75


In [41]:
%%latex
$$gower(\vec{a}, \vec{b})=\dfrac{\sum_{i=1}^{n}w_i * d(a_i, b_i)}{\sum_{i=1}^{n}w_i}$$
$$d(a,b) = \begin{cases}
\dfrac{|a-b|}{max(x_i) - min_i(x_i)}  & \text{if variable at $i$ is ordinal/continous} \\
\begin{cases}
0, & \text{if } a\neq b \\
1, & \text{if } a = b
\end{cases} & \text{if variable at $i$ is categorical}
\end{cases}$$

<IPython.core.display.Latex object>

In [45]:
from ldap3 import Server, Connection, ALL
server = Server('ldap.psu.edu', get_info=ALL)
conn = Connection(server, 'dc=psu,dc=edu')
#conn.search('dc=demo1,dc=freeipa,dc=org', '(objectclass=person)')
#conn.entries